In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

print(pd.__version__)

%matplotlib inline

0.22.0


In [17]:
train_df = pd.read_table("./train.tsv", index_col = 'train_id', dtype = {'item_condition_id':CategoricalDtype(categories = [str(i) for i in range(1,6)], ordered = True), 'category_name':'category', 'brand_name': 'category', 'shipping':'category'})

/Users/Abe/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
a, b, c, d = train_df['category_name'].str.split("/", 3).str
category_oneLevel = a
category_twoLevel = a + "/" + b
category_threeLevel = a + "/" + b + "/" + c
train_df['category_oneLevel'] = category_oneLevel
train_df['category_twoLevel'] = category_twoLevel
train_df['category_threeLevel'] = category_threeLevel
for col in ['category_oneLevel', 'category_twoLevel', 'category_threeLevel']:
    train_df[col] = train_df[col].astype('category')

In [30]:
for i in train_df.category_oneLevel.cat.categories.values:
    print(i)

Beauty
Electronics
Handmade
Home
Kids
Men
Other
Sports & Outdoors
Vintage & Collectibles
Women


In [31]:
for i in train_df.category_twoLevel.cat.categories.values:
    print(i)

Beauty/Bath & Body
Beauty/Fragrance
Beauty/Hair Care
Beauty/Makeup
Beauty/Other
Beauty/Skin Care
Beauty/Tools & Accessories
Electronics/Cameras & Photography
Electronics/Car Audio, Video & GPS
Electronics/Cell Phones & Accessories
Electronics/Computers & Tablets
Electronics/Media
Electronics/Other
Electronics/TV, Audio & Surveillance
Electronics/Video Games & Consoles
Handmade/Accessories
Handmade/Art
Handmade/Bags and Purses
Handmade/Books and Zines
Handmade/Candles
Handmade/Ceramics and Pottery
Handmade/Children
Handmade/Clothing
Handmade/Crochet
Handmade/Dolls and Miniatures
Handmade/Furniture
Handmade/Geekery
Handmade/Glass
Handmade/Holidays
Handmade/Housewares
Handmade/Jewelry
Handmade/Knitting
Handmade/Music
Handmade/Needlecraft
Handmade/Other
Handmade/Others
Handmade/Paper Goods
Handmade/Patterns
Handmade/Pets
Handmade/Quilts
Handmade/Toys
Handmade/Weddings
Handmade/Woodworking
Home/Artwork
Home/Bath
Home/Bedding
Home/Cleaning Supplies
Home/Furniture
Home/Home Appliances
Home/Ho

In [32]:
# Create column for (adult) clothing type that does not distinguish gender - You will probably just have NaN's for all the items that are not clothing